# iLOrest examples


Version 0.32

## Introduction

This Jupyter Notebook contains iLOrest examples as well as tips and tricks answering recurrent questions from HPE customers.

## List of examples

The following GET and SET examples are present in this Jupyter Notebook. Note that this list is growing overtime. The classification is not following a strict model; it is a mix of Root service and data type locations.

HPE OEM extensions are TBD

* AccountService
   - [iLO users: Enforce password complexity and minimum password length](#EnforceiLOUsersPasswordComplexity)

* Chassis
   - [ThermalConfiguration](#ThermalConfiguration)
    
* Managers (iLO)
    - [Enable/Disable SNMPv1 in iLO](#SNMPv1Enabled)
 
* Resource Directory
    - TBD

* CompusterSystem
     - [PostState and DiscoveryComplete](#PostState_And_DiscoveryComplete)
     - [Bios/RBSU Login password](#ManageBiosRbsuPassword)

## Environment preparation

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [1]:
########## Environment preparation ((Version: 0.1)) ############


# Set Student ID number
Stud=00

# location and ports variables
let OpenBmcPort=44000+${Stud}
let iLO5SimulatorPort=45000+${Stud}
let ilo5Port=443

CacheDir="${PWD}/iLOrestCache"
LogDir="${PWD}"
LogFile="${LogDir}/iLOrest.log"

iLO5SimulatorIP=bmcsimulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

OpenBmcIP=bmcsimulators
OpenBmc="${OpenBmcIP}:${OpenBmcPort}"
OpenBmcURI="https://${OpenBmc}"

ilo5IP="ilo5"
ilo5="${ilo5IP}:${ilo5Port}"
ilo5URI="https://${ilo5}"

# Credentials
User="student"
Password='P@ssw0rd!'

# Miscellaneous
alias ilorest='ilorest --nologo --cache-dir ${CacheDir} --logdir ${LogDir}'
ilorest logout &>/dev/null # Clear cache


# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

ilo5 is reachable
OpenBmc is reachable
iLO5Simulator is reachable


### iLO 5 login and cache redirection

This infrastructure contains only one physical computer and prevents multiple students to perform SET actions on this unique system. Therefore, students have to use a DMTF BMC simulator populated with an iLO 5 mockup. However, because iLOrest populates its cache directory during the login process you need first to authenticate with the iLO 5 and then modify the cache files to have them pointing to the DMTF simulator. 

In [2]:
ilorest login $ilo5 -u $User -p $Password

# Redirect cache toward the iLO 5 Simulator
sed -i "s.${ilo5URI}.${iLO5SimulatorURI}." ${CacheDir}/cache/*

Discovering data...Done


## AccountService

This root service models the iLO User Accounts.

<a id = "EnforceiLOUsersPasswordComplexity">iLO users: Enforce password complexity and minimum password length</a>\
API Reference document keyword: `EnforcePasswordComplexity` \
iLO GUI location: `Security - Access Settings`

In [3]:
# Select data type
# ilorest select HpeiLOAccountService # TODO/TBD: populate ResourceDirectory with HpeiLOAccountService 
ilorest select AccountService


ilorest list Oem/Hpe --json

 
ilorest set Oem/Hpe/EnforcePasswordComplexity=True
ilorest set Oem/Hpe/MinPasswordLength=5

ilorest status
#ilorest commit

{
  "Oem": {
    "Hpe": {
      "@odata.context": "/redfish/v1/$metadata#HpeiLOAccountService.HpeiLOAccountService", 
      "@odata.id": "/redfish/v1/AccountService/", 
      "@odata.type": "#HpeiLOAccountService.v2_3_0.HpeiLOAccountService", 
      "Actions": {
        "#HpeiLOAccountService.ImportKerberosKeytab": {
          "target": "/redfish/v1/AccountService/Actions/Oem/Hpe/HpeiLOAccountService.ImportKerberosKeytab/"
        }
      }, 
      "AuthFailureDelayTimeSeconds": 10, 
      "AuthFailureLoggingThreshold": 3, 
      "AuthFailuresBeforeDelay": 1, 
      "DefaultPassword": null, 
      "DefaultUserName": null, 
      "DirectorySettings": {
        "LdapAuthenticationMode": "Disabled", 
        "LdapCaCertificateLoaded": false, 
        "LdapCaCertificates": {
          "@odata.id": "/redfish/v1/AccountService/ExternalAccountProviders/LDAP/Certificates/"
        }, 
        "LdapServerPort": 636
      }, 
      "DirectoryTest": {
        "@odata.id": "/redfish/v1/AccountServ

## Chassis

This section contains parameters related to the `Chassis`data type as well as data types under the Oem extensions below this data type.

### <a id = "ThermalConfiguration">ThermalConfiguration</a>

> **NOTE** This parameter is new with iLO 5 firmware 2.30. 

    API Reference document keyword: `TBD: Not yet documented` \
    iLO GUI location: `TBD: Not yet implemented`

In [4]:
# Select data type
ilorest select HpeThermalExt

ilorest list --json


Unable to locate instance for hpethermalext
{
  "@odata.context": "/redfish/v1/$metadata#AccountService.AccountService", 
  "@odata.etag": "W/\"8F1B1B4B\"", 
  "@odata.id": "/redfish/v1/AccountService/", 
  "@odata.type": "#AccountService.v1_5_0.AccountService", 
  "Accounts": {
    "@odata.id": "/redfish/v1/AccountService/Accounts/"
  }, 
  "ActiveDirectory": {
    "AccountProviderType": "ActiveDirectoryService", 
    "Authentication": {
      "AuthenticationType": "UsernameAndPassword", 
      "Password": null, 
      "Username": ""
    }, 
    "RemoteRoleMapping": [
      {
        "LocalRole": "dirgroupb3d8954f6ebbe735764e9f7c", 
        "RemoteGroup": "Administrators"
      }, 
      {
        "LocalRole": "dirgroup9d4546a03a03bb977c03086a", 
        "RemoteGroup": "Authenticated Users:S-1-5-11"
      }
    ], 
    "ServiceAddresses": [
      ""
    ], 
    "ServiceEnabled": false
  }, 
  "Description": "iLO User Accounts", 
  "Id": "AccountService", 
  "LDAP": {
    "AccountProvi

## Managers

Examples in this section are related to the `Manager` data type as well as data Oem data types located below `/redfish/v1/Managers/` end point like `HpeiLOSnmpService`. 

### <a id = "SNMPv1Enabled">Enable/Disable SNMPv1 in iLO</a>

> **NOTE** Changing the `SNMPv1Enabled` setting appeared in iLO 5 FW 2.30.

    API Reference document keyword: `SNMPv1Enabled` \
    iLO GUI location: `TBD`

In [5]:
# Select data type
ilorest select HpeiLOSnmpService

# Get current value
ilorest get SNMPv1Enabled
echo

# Set new value
ilorest set SNMPv1Enabled=False

ilorest status

SNMPv1Enabled=False

ERROR	: Setting for 'snmpv1enabled' is the same as the current value.
Current changes found:
AccountService.v1_5_0(/redfish/v1/AccountService/)
	Oem/Hpe/EnforcePasswordComplexity=True
	Oem/Hpe/MinPasswordLength=5


## HPE iLO Resource Directory

The HPE iLO Resource Directory is an HPE Oem extention containing the exhaustive list of the data types present the in a Redfish pecific data types. This directory can be used by Redfish clients to find faster find a specific resource in the Redfish tree.

In [6]:
Type="HpeThermalExt"
#Type="SoftwareInventory"
ilorest types | grep Resource
ilorest select HpeiLOResourceDirectory
echo
ilorest list instances --json | jq -r '.[]|.[]|."@odata.type"' \
         | awk -F. '/'${Type}'/ {print $1}' | sort -u

## ComputerSystem

This section contains examples related to the `ComputerSystem` data type.

### <a id = "PostState_And_DiscoveryComplete">PostState and DiscoveryComplete</a>

Use these state parameters to evaluate the current state of a system before modifying or viewing its resources. As explained in this [article](https://developer.hpe.com/blog/master-the-redfish-server-states-to-improve-your-monitoring-and-manageme) certain parameters cannot be modified if the computer is not in a specific state.

Moreover, some parameters cannot be viewed when the entire discovery of all subsystems is not complete. As an example, you will not be able to view or modify SmartArray and logical drive configurations if the `SmartArrayDiscovery` is not complete.

In [7]:
ilorest select ComputerSystem
# Try to include HpeComputerSystemExt in the Mockup 
ilorest get Oem/Hpe/DeviceDiscoveryComplete
echo
ilorest get Oem/Hpe/PostState 
echo 


Oem=
     Hpe=
          DeviceDiscoveryComplete=
                                   AMSDeviceDiscovery=Complete
                                   DeviceDiscovery=vMainDeviceDiscoveryComplete
                                   SmartArrayDiscovery=Complete

Oem=
     Hpe=
          PostState=FinishedPost



### <a id = "ManageBiosRbsuPassword">Bios/RBSU Login password</a>

> NOTE: requires iLOrest >= 3.1.1

In [9]:
# Setting a password for the first time
ilorest setpassword FirstTimePassword '""'

# Removing the Bios/RBSU password
ilorest setpassword '""' FirstTimePassowrd

# No commit needed. No iLO nor system reset needed. 
ilorest status


An invalid response body was returned: No JSON object could be decodedNo error message returned or unable to parse error response.
An invalid response body was returned: No JSON object could be decodedNo error message returned or unable to parse error response.
Current changes found:
AccountService.v1_5_0(/redfish/v1/AccountService/)
	Oem/Hpe/EnforcePasswordComplexity=True
	Oem/Hpe/MinPasswordLength=5


## Thermal

In [ ]:
ilorest serverinfo --thermal
ilorest list --json

In [10]:
ilorest logout

Logging session out.


## TBD
